# <font color='blue'>Traductor de idiomas con aprendizaje automático y NLP</font>

In [0]:
# Versión del lenguaje Python
# from platform import python_version
print('Versión del lenguaje Python utilizada en este Jupyter Notebook:', python_version())

Versión del lenguaje Python utilizada en este Jupyter Notebook: 3.9.5


In [0]:
# Para actualizar un paquete, ejecute el siguiente comando en una terminal o símbolo del sistema:
# pip install -U nombre_paquete

# Para instalar la versión exacta de un paquete, ejecute el siguiente comando en una terminal o símbolo del sistema:
# !pip instalar antorcha==1.5.0

# Después de instalar o actualizar el paquete, reinicie el cuaderno jupyter.

# Instale el paquete de marca de agua.
# Este paquete se usa para escribir versiones de otros paquetes usados ​​en este cuaderno jupyter.
!pip install -q -U watermark

In [0]:
# Instalar PyTorch
!pip install -q torch 

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-d78c7c25-a401-4b32-ab5e-4bd2dcb2a79d/bin/python -m pip install --upgrade pip' command.


In [0]:
# El paquete torchtext proporciona varios conjuntos de datos y funciones para NLP
# https://torchtext.readthedocs.io/en/latest/index.html
!pip install -q torchtext

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cpu requires torch==1.12.1, but you have torch 2.0.1 which is incompatible.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-d78c7c25-a401-4b32-ab5e-4bd2dcb2a79d/bin/python -m pip install --upgrade pip' command.


In [0]:
# Instalar o spacy
!pip install -q spacy

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-d78c7c25-a401-4b32-ab5e-4bd2dcb2a79d/bin/python -m pip install --upgrade pip' command.


In [0]:
# Imports
import math
import time
import spacy
import torch
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torchtext
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

---------------------------------------------------------------------------
ImportError                               Traceback (most recent call last)
<command-3352856648243565> in <cell line: 12>()
     10 import torchtext
     11 from torchtext.datasets import Multi30k
---> 12 from torchtext.data import Field, BucketIterator

ImportError: cannot import name 'Field' from 'torchtext.data' (/local_disk0/.ephemeral_nfs/envs/pythonEnv-d78c7c25-a401-4b32-ab5e-4bd2dcb2a79d/lib/python3.9/site-packages/torchtext/data/__init__.py)

In [0]:
# Versiones de paquetes utilizados en este cuaderno jupyter
%reload_ext watermark
%watermark -a "Eric Passos" --iversions

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
<command-3352856648243566> in <cell line: 2>()
      1 # Versões dos pacotes usados neste jupyter notebook
----> 2 get_ipython().run_line_magic('reload_ext', 'watermark')
      3 get_ipython().run_line_magic('watermark', '-a "Eric Passos" --iversions')

/databricks/python/lib/python3.9/site-packages/IPython/core/interactiveshell.py in run_line_magic(self, magic_name, line, _stack_depth)
   2405                 kwargs['local_ns'] = self.get_local_scope(stack_depth)
   2406             with self.builtin_trap:
-> 2407                 result = fn(*args, **kwargs)
   2408             return result
   2409 

/databricks/python/lib/python3.9/site-packages/decorator.py in fun(*args, **kw)
    230             if not kwsyntax:
    231                 args, kw = fix(args, kw, sig)
--> 232             return caller(func, *(extras + args), **kw)
   

In [0]:
# Aquí definimos el dispositivo que se usará para entrenar el modelo
# Si hay al menos una GPU disponible, usaremos el dispositivo 'cuda' (nombre de plataforma Nvidia para GPU)
# Si no hay GPU, usaremos CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

El siguiente comando funcionará solo si la plataforma CUDA de Nivida está instalada en la computadora. Si quieres saber más sobre la plataforma CUDA, visita aquí:
https://developer.nvidia.com/cuda-toolkit

In [0]:
# GPU en el servidor
!nvidia-smi

Thu May 21 21:49:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN X (Pascal)    On   | 00000000:05:00.0 Off |                  N/A |
| 23%   41C    P8     9W / 250W |    125MiB / 12194MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  On   | 00000000:09:00.0 Off |                  N/A |
| 23%   38C    P8     9W / 250W |     12MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

### Diccionarios

Necesitamos instalar los diccionarios de idioma que se utilizarán para entrenar el modelo. Aquí puede encontrar detalles sobre los conjuntos de datos:

https://www.statmt.org/wmt16/multimodal-task.html

In [0]:
# Descarga el diccionario de inglés
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/dmpm/anaconda3/lib/python3.7/site-packages/en_core_web_sm -->
/home/dmpm/anaconda3/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [0]:
# Descarga el diccionario alemán
 !python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/home/dmpm/anaconda3/lib/python3.7/site-packages/de_core_news_sm -->
/home/dmpm/anaconda3/lib/python3.7/site-packages/spacy/data/de
You can now load the model via spacy.load('de')


Ahora cargamos los diccionarios en la memoria.

In [0]:
# Carregando os dicionários
spacy_german = spacy.load('de')
spacy_english = spacy.load('en')

Vamos a crear dos funciones para tokenizar los diccionarios.

In [0]:
# Función para tokenizar el diccionario de inglés
def tokenize_english(text):
    return [token.text for token in spacy_english.tokenizer(text)][::-1]

In [0]:
# función de tokenización del diccionario alemán
def tokenize_german(text):
    return [token.text for token in spacy_german.tokenizer(text)]

Ahora necesitamos crear el origen y el destino, es decir, el idioma de origen y el idioma de destino para nuestro traductor.

Nuestro modelo debería traducirse del inglés al alemán. El inglés será la fuente (SOURCE) y el alemán será el destino (TARGET).

In [0]:
# Lenguaje fuente
SOURCE = Field(tokenize = tokenize_english, init_token = '<sos>', eos_token = '<eos>', lower = True)

In [0]:
# Lengua de llegada
TARGET = Field(tokenize = tokenize_german, init_token = '<sos>', eos_token = '<eos>', lower = True)

In [0]:
# Usamos split() del paquete Multi30k de torchtext para dividir los diccionarios en SOURCE y TARGET
# y luego en entrenamiento, validación y prueba
# Nota: Los datos se descargarán en el paquete Multi30k
dados_treino, dados_valid, dados_teste = Multi30k.splits(exts = ('.en', '.de'), fields = (SOURCE, TARGET))

In [0]:
# Visualización de datos de entrenamiento, Source y Target
print(dados_treino.examples[0].src)
print(dados_treino.examples[0].trg)

['.', 'bushes', 'many', 'near', 'outside', 'are', 'males', 'white', ',', 'young', 'two']
['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.']


In [0]:
print("Tamaño del conjunto de datos de entrenamiento: " + str(len(dados_treino.examples)))
print("Tamaño del conjunto de datos de validación: " + str(len(dados_valid.examples)))
print("Tamaño del conjunto de datos de prueba: " + str(len(dados_teste.examples)))

Tamanho do Dataset de Treino: 29000
Tamanho do Dataset de Validação: 1014
Tamanho do Dataset de Teste: 1000


In [0]:
# Vamos a crear los vocabularios SOURCE y TARGET
SOURCE.build_vocab(dados_treino, min_freq = 2)
TARGET.build_vocab(dados_treino, min_freq = 2)

In [0]:
# Imprimir el tamaño de los vocabularios
print("Tamaño del vocabulario en inglés (SOURCE): " + str(len(SOURCE.vocab)))
print("Tamaño del vocabulario alemán (TARGET): " + str(len(TARGET.vocab)))

Tamanho do Vocabulário em Inglês (SOURCE): 5893
Tamanho do Vocabulário em Alemão (TARGET): 7855


### Construcción del modelo

Crearemos 3 clases:

- Codificador
- decodificador
- Seq2Seq

In [0]:
# Clase para el codificador
class Encoder(nn.Module):
    
    # Método constructor
    def __init__(self, input_dims, emb_dims, hid_dims, n_layers, dropout):
        super().__init__()
        
        # Capas de modelo
        self.hid_dims = hid_dims
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dims, emb_dims)
        self.rnn = nn.LSTM(emb_dims, hid_dims, n_layers, dropout = dropout)
        self.dropout = nn.Dropout(dropout)
        
    # Método forward para entrenamiento
    def forward(self, src):
        
        # Ejecución del modelo
        embedded = self.dropout(self.embedding(src))
        outputs, (h, cell) = self.rnn(embedded)
        
        return h, cell

In [0]:
# Clase para el decodificador
class Decoder(nn.Module):
    
    # Método constructor
    def __init__(self, output_dims, emb_dims, hid_dims, n_layers, dropout):
        super().__init__()
        
        # Camadas do modelo
        self.output_dims = output_dims
        self.hid_dims = hid_dims
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dims, emb_dims)
        self.rnn = nn.LSTM(emb_dims, hid_dims, n_layers, dropout = dropout)
        self.fc_out = nn.Linear(hid_dims, output_dims)
        self.dropout = nn.Dropout(dropout)
        
    # Método forward  para entrenamiento
    def forward(self, input, h, cell):
              
        # Ejecución del modelo
        input = input.unsqueeze(0)   
        embedded = self.dropout(self.embedding(input))     
        output, (h, cell) = self.rnn(embedded, (h, cell))
        pred = self.fc_out(output.squeeze(0))
        
        return pred, h, cell

In [0]:
# Clase para el modelo Seq2Seq
class Seq2Seq(nn.Module):
    
    # método constructor
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        # Capas de modelo
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    # Método forward para entrenamiento
    def forward(self, src, trg, teacher_forcing_rate = 0.5):
        
        # Ejecución del modelo
        batch_size = trg.shape[1]
        target_length = trg.shape[0]
        target_vocab_size = self.decoder.output_dims
        outputs = torch.zeros(target_length, batch_size, target_vocab_size).to(self.device)
        h, cell = self.encoder(src)
        input = trg[0,:]
        
        for t in range(1, target_length):

            output, h, cell = self.decoder(input, h, cell)
            outputs[t] = output
            top = output.argmax(1) 
            input = trg[t] if (random.random() < teacher_forcing_rate) else top
        
        return outputs

Definamos algunos hiperparámetros y los datos generados.

In [0]:
# Hiperparámetros
batch_size = 32
input_dimensions = len(SOURCE.vocab)
output_dimensions = len(TARGET.vocab)
encoder_embedding_dimensions = 256
decoder_embedding_dimensions = 256
hidden_layer_dimensions = 512
num_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
epochs = 20
grad_clip = 1
lowest_validation_loss = float('inf')

In [0]:
# Heneradores de datos
iterador_treino, iterador_valid, iterador_teste = BucketIterator.splits((dados_treino, dados_valid, dados_teste), 
                                                                        batch_size = batch_size, 
                                                                        device = device)

Aquí creamos el codificador, decodificador y modelo:

In [0]:
# Instancia de codificador
encod = Encoder(input_dimensions, 
                encoder_embedding_dimensions,
                hidden_layer_dimensions, 
                num_layers, 
                encoder_dropout)

In [0]:
# Instancia de decodificador
decod = Decoder(output_dimensions, 
                decoder_embedding_dimensions,
                hidden_layer_dimensions, 
                num_layers, 
                decoder_dropout)

In [0]:
# Instancia de modelo
modelo = Seq2Seq(encod, decod, device).to(device)

In [0]:
# Modelo creado
modelo

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=7855, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

Definamos la función de inicialización de pesos, la función de costo y el optimizador.

In [0]:
# Necesitamos una función para inicializar los pesos de las redes neuronales
def inicializa_pesos(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.1, 0.1)

In [0]:
# Incluimos la función de inicialización del peso en el modelo.
modelo.apply(inicializa_pesos)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=7855, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [0]:
# Definimos la función de coste para calcular el error del modelo
criterion = nn.CrossEntropyLoss(ignore_index = TARGET.vocab.stoi[TARGET.pad_token])

In [0]:
# Creamos el optimizador para actualizar los pesos del modelo con cada pase de entrenamiento
optimizer = optim.Adam(modelo.parameters())

Si bien no es obligatorio, la creación de funciones para la capacitación y evaluación de modelos ayuda a modularizar nuestro proceso de capacitación de modelos.

In [0]:
# Función para entrenar el modelo.
def treina_modelo(modelo, iterator, optimizer, criterion, clip):
    
    # Iniciar el método de entrenamiento
    modelo.train()
    
    # Inicializar error de época
    epoch_loss = 0
    
    # Iterador de bucle (generador de datos)
    for i, batch in enumerate(iterator):
        
        # Recopilamos datos de origen y destino
        src = batch.src
        trg = batch.trg
        
        # Reseteamos los gradientes
        optimizer.zero_grad()
        
        # Hacemos predicciones con el modelo
        output = modelo(src, trg)
        
        # Ajustamos la forma de las previsiones
        output_dims = output.shape[-1]
        output = output[1:].view(-1, output_dims)
        trg = trg[1:].view(-1)
        
        # Calculamos el error del modelo
        loss = criterion(output, trg)
        
        # Empezamos la backpropgation
        loss.backward()
        
        # Calculamos los gradientes de la derivada para actualizar los pesos
        torch.nn.utils.clip_grad_norm_(modelo.parameters(), clip)
        
        # Aplicamos la actualización de peso
        optimizer.step()
        
        # Almacenamos el error de época
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [0]:
# Función para evaluar el modelo.
def avalia_modelo(modelo, iterator, criterion):
    
    # Iniciar la función de evaluación
    modelo.eval()
    
    # Inicializar error de época
    epoch_loss = 0
    
    # Hagamos predicciones con el modelo.
    with torch.no_grad():
    
        # Iterador de bucle (generador de datos)
        for i, batch in enumerate(iterator):

            # Extraer origen y destino
            src = batch.src
            trg = batch.trg

            # Pronóstico con el modelo
            output = modelo(src, trg, 0)

            # Ajusta las dimensiones de las previsiones
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            # Calcula el error del modelo
            loss = criterion(output, trg)
            
            # Almacenar el error en la época.
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

### Entrenamiento del modelo

La formación de modelos requiere mucho tiempo. Sea paciente.

In [0]:
# Recorra el número de épocas para entrenar el modelo
for epoch in range(epochs):
    
    # Registrar la hora en que empezamos
    start_time = time.time()
    
    # Capacitación
    train_loss = treina_modelo(modelo, iterador_treino, optimizer, criterion, grad_clip)
    
    # Validación
    valid_loss = avalia_modelo(modelo, iterador_valid, criterion)
    
    # Registra el tiempo cuando terminamos
    end_time = time.time()
    
    # Verificamos el error mínimo y luego guardamos el modelo al verificar el modelo con mejor rendimiento.
    if valid_loss < lowest_validation_loss:
        lowest_validation_loss = valid_loss
        torch.save(modelo.state_dict(), 'modelos/seq2seq.pt')
    
    # Print
    print(f'Epoch: {epoch+1:02} | Time: {np.round(end_time-start_time,0)}s')
    print(f'\tErro em Treino: {train_loss:.4f}')
    print(f'\t Erro em Validação: {valid_loss:.4f}')

Epoch: 01 | Time: 48.0s
	Erro em Treino: 4.7076
	 Erro em Validação: 4.6512
Epoch: 02 | Time: 47.0s
	Erro em Treino: 3.9973
	 Erro em Validação: 4.3468
Epoch: 03 | Time: 47.0s
	Erro em Treino: 3.6368
	 Erro em Validação: 4.0514
Epoch: 04 | Time: 48.0s
	Erro em Treino: 3.3807
	 Erro em Validação: 3.8883
Epoch: 05 | Time: 48.0s
	Erro em Treino: 3.1684
	 Erro em Validação: 3.8140
Epoch: 06 | Time: 48.0s
	Erro em Treino: 2.9915
	 Erro em Validação: 3.6792
Epoch: 07 | Time: 47.0s
	Erro em Treino: 2.8233
	 Erro em Validação: 3.6726
Epoch: 08 | Time: 48.0s
	Erro em Treino: 2.6837
	 Erro em Validação: 3.6291
Epoch: 09 | Time: 48.0s
	Erro em Treino: 2.5523
	 Erro em Validação: 3.6190
Epoch: 10 | Time: 47.0s
	Erro em Treino: 2.4393
	 Erro em Validação: 3.5998
Epoch: 11 | Time: 47.0s
	Erro em Treino: 2.3307
	 Erro em Validação: 3.5683
Epoch: 12 | Time: 48.0s
	Erro em Treino: 2.2307
	 Erro em Validação: 3.5983
Epoch: 13 | Time: 48.0s
	Erro em Treino: 2.1253
	 Erro em Validação: 3.5880
Epoch: 14 | 

### Evaluación del modelo

Con el modelo entrenado, lo evaluamos con datos de prueba.

In [0]:
# Cargamos el modelo entrenado
modelo.load_state_dict(torch.load('modelos/seq2seq.pt'))

<All keys matched successfully>

In [0]:
# Evaluamos el modelo
test_loss = avalia_modelo(modelo, iterador_teste, criterion)

In [0]:
# Print
print(f'Erro em Teste: {test_loss:.4f}')

Erro em Teste: 3.5024


### Traduciendo Idioma

Modelo entrenado y evaluado, usémoslo para el propósito para el cual fue creado.

In [0]:
# Función para traducción de idiomas en 5 oraciones.
def traduz_idioma(modelo, iterator, limit = 5):
    
    with torch.no_grad():
    
        # Bucle a través del iterador
        for i, batch in enumerate(iterator):
            
            # Mientras estemos dentro del límite, traduciremos
            if i < limit :
                
                # Extraemos SOURCE y TARGET
                # Hacemos esto para poder comparar la traducción correcta con la predicción.
                src = batch.src
                trg = batch.trg

                # Modelo de predicción
                output = modelo(src, trg, 0)
                
                # Todas las predicciones
                preds = torch.tensor([[torch.argmax(x).item()] for x in output])
                
                # Prints
                print('Texto de Origem em Inglês: ' + str([SOURCE.vocab.itos[x] for x in src][1:-1][::-1]))
                print('Texto de Destino em Alemão (Valor Esperado): ' + str([TARGET.vocab.itos[x] for x in trg][1:-1]))
                print('Texto de Destino em Alemão (Valor Previsto): ' + str([TARGET.vocab.itos[x] for x in preds][1:-1]))
                print('\n')

In [0]:
# Generemos texto aleatorio a partir de los datos disponibles.
_, _, iterador_translate = BucketIterator.splits((dados_treino, dados_valid, dados_teste), 
                                                 batch_size = 1, 
                                                 device = device)

In [0]:
# Traducción de idiomas
saida = traduz_idioma(modelo, iterador_translate)

Texto de Origem em Inglês: ['two', 'men', 'wearing', 'hats', '.']
Texto de Destino em Alemão (Valor Esperado): ['zwei', 'männer', 'mit', 'mützen', '.']
Texto de Destino em Alemão (Valor Previsto): ['zwei', 'männer', 'mit', 'schwarzen', 'haaren']


Texto de Origem em Inglês: ['young', 'woman', 'climbing', 'rock', 'face']
Texto de Destino em Alemão (Valor Esperado): ['junge', 'frau', 'klettert', 'auf', 'felswand']
Texto de Destino em Alemão (Valor Previsto): ['eine', 'junge', 'frau', 'klettert', 'eine']


Texto de Origem em Inglês: ['a', 'woman', 'is', 'playing', 'volleyball', '.']
Texto de Destino em Alemão (Valor Esperado): ['eine', 'frau', 'spielt', 'volleyball', '.']
Texto de Destino em Alemão (Valor Previsto): ['eine', 'frau', 'spielt', 'volleyball', '.']


Texto de Origem em Inglês: ['three', 'men', 'are', 'walking', 'up', 'hill', '.']
Texto de Destino em Alemão (Valor Esperado): ['drei', 'männer', 'gehen', 'bergauf', '.']
Texto de Destino em Alemão (Valor Previsto): ['drei', 'männ

¡Felicidades! Ahí lo tienes, tu traductor de textos con Machine Learning y NLP.

## Fin